# Exercises

## Jupyter notebook usage

- To execute a code block (to run a cell) hit: `shift + return` or click on the `Run` button at the top.
- After execution the circle in the top-right corner is filled grey. Once execution has finished the circle will be filled white again.

## Exercises 1 - ohsome API - Aggregation

Question: What is the temporal evolution of the number of hospitals in Florence since 2014?

To answer this question we need to find out the right ohsome API endpoint, how to define the boundary (AOI), the OSM tags to be used as filter and format of the time parameter. Below a list of links is provided to read up on all those topics.

- [ohsome API - Endpoints (elements aggregation)](https://docs.ohsome.org/ohsome-api/v1/endpoints.html#elements-aggregation)
- Boundaries:
    - [ohsome API - Boundaries](https://docs.ohsome.org/ohsome-api/v1/boundaries.html)
    - [OSM Wiki - Bounding Box](https://wiki.openstreetmap.org/wiki/Bounding_Box) 
    - Florence: `11.1645, 43.7034, 11.3242, 43.8579`
- Filter (OSM Tags): 
    - [OSM Wiki - Healthcare](https://wiki.openstreetmap.org/wiki/Healthcare)
    - [ohsome API - Filter](https://docs.ohsome.org/ohsome-api/v1/filter.html)
    - OSM tags describing a hospital:
        - `amenity=hospital`
        - `healthcare=hospital`
- Time:
    - [ohsome API - Time](https://docs.ohsome.org/ohsome-api/stable/time.html)

### Request the API 

Please fill in the empty strings marked with a `TODO` comment.

In [2]:
import json

from datetime import datetime

import requests


base_url = "https://api.ohsome.org/v1"
endpoint = ""  # TODO
url = base_url + endpoint
parameters = {  # TODO
    "bboxes": "",
    "filter": "",
    "format": "",
    "time": "",
}
headers = {
    "accept": "application/json",
    "Content-Type": "application/x-www-form-urlencoded",
}

for value in parameters.values():
    assert value != "", "Please provide values for the parameters"
    
response = requests.post(url, data=parameters, headers=headers)
response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

print("Response:")
print(json.dumps(response.json(), indent=4))  # Pretty print response

result = response.json()["result"]

AssertionError: Please provide values for the parameters

### Plot the results

Once the numbers of hospitals are requested run below code to plot the results.

In [3]:
import matplotlib.pyplot as plt


dates = []
values = []
for result in results:
    year = datetime.fromisoformat(result["timestamp"].replace("Z", ""))
    dates.append(year)
    values.append(result["value"])

fig, ax = plt.subplots()
ax.plot(dates, values, label="Number of Hospitals")
ax.legend()
fig.suptitle("Temporal Evolution of Hospitals")
plt.show()

NameError: name 'results' is not defined

## Exercise 2 - ohsome API - Data extraction

Question: Where were schools mapped over the years?

Goal: Get and visualize the state of schools in OSM on a yearly basis since 2014.

To answer this question we need to get the features representing schools as point from the ohsome API.

- Read up on data extraction here: https://api.ohsome.org/v1/swagger-ui.html?urls.primaryName=Data%20Extraction
- Tags for schools in OSM are described here: https://wiki.openstreetmap.org/wiki/Tag:amenity%3Dschool

### Request the ohsome API

Please fill in the empty strings marked with a `TODO` comment.

In [4]:
import json

import requests


base_url = "https://api.ohsome.org/v1"
endpoint = ""  # TODO
url = base_url + endpoint

with open("italy-florence.geojson", "r") as file:
    bpolys = json.load(file)

parameters = {  # TODO
    "bpolys": json.dumps(bpolys),
    "filter": "",
    "time": "",
}
headers = {
    "accept": "application/json",
    "Content-Type": "application/x-www-form-urlencoded",
}

for value in parameters.values():
    assert value != "", "Please provide values for the parameters"

response = requests.post(url, data=parameters, headers=headers)
response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

feature_collection = response.json()

AssertionError: Please provide values for the parameters

## Display on a map

In [5]:
import folium

from folium.plugins import TimestampedGeoJson


m = folium.Map()
m.fit_bounds([(43.7034, 11.1645), ( 43.8579, 11.3242,)])

# Add admin boundary to map
with open("italy-florence.geojson", "r") as file:
    admin_boundary = json.load(file)
folium.GeoJson(admin_boundary, name="Admin Boundary").add_to(m)

# Rename property "time"
feature_collection = response.json()
for feature in feature_collection["features"]:
    feature["properties"]["time"] = feature["properties"].pop("@snapshotTimestamp")
    
# Add centroids to map
TimestampedGeoJson(feature_collection, period="P1Y").add_to(m)
m.save("index.html")

# Show map
m

NameError: name 'response' is not defined

## Exercise 3 - OQT API - Data Quality

OQT provides quality estimations of OpenStreetMap (OSM) data.
OQT calls those estimations Indicators.
Indicators are computed for a specific area and a certain set of aggregated OSM features called Layer. Basically a layer defines the request to the ohsome API.

Two intrinsic quality estimation OQT offers are the Mapping Saturation indicator and the Currentness indicator.

### Request the Mapping Saturation Indicator from the OQT API

The Mapping Saturation indicator calculate the saturation of mapping activity within the last 3 years.
It is based on the premise that each aggregation of features (e.g. length of roads or count of buildings) has a maximum. After increased mapping activity saturation is reached near this maximum.
The Mapping Saturation indicator works well with following layers:

- Building Count (`building_count`)
- Major Roads Length (`major_roads_length`)
- Amenities (`amenities`)
- Point-of-Interests (`poi`)

#### Task

Choose a layer you find interesting and request the Mapping Saturation Indicator from the OQT API.

In [7]:
import json

import requests
from IPython.display import SVG, display


base_url = "https://oqt.ohsome.org/api"
endpoint = "/indicator"
url = base_url + endpoint

# OQT size restriction on the input geometry is 100 sqkm
with open("italy-florence-quartier-5.geojson", "r") as file:
    bpolys = json.load(file)

parameters = {
    "name": "MappingSaturation",
    "layerName": "",  # TODO
    "bpolys": bpolys,
    "flatten": False,
    "includeSvg": True,
}

for value in parameters.values():
    assert value != "", "Please provide values for the parameters"

response = requests.post(url, json=parameters)
response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

properties = response.json()["properties"]
result = properties["result"]
svg = result.pop("svg")

print("Property keys: ")
print(properties.keys())
print("")
print("Result: ")
print(json.dumps(result, indent=4))  # Pretty print response

display(SVG(svg))

AssertionError: Please provide values for the parameters

### Request the Currentness Indicator from the OQT AP

The Currentness indicator works well with following layers:

- Building Count (`building_count`)
- Major Roads Count (`major_roads_count`)
- Amenities (`amenities`)

#### Task

Choose a layer you find interesting and request the Mapping Saturation Indicator from the OQT API.

In [8]:
import json

import requests
from IPython.display import SVG, display


base_url = "https://oqt.ohsome.org/api"
endpoint = "/indicator"
url = base_url + endpoint

with open("italy-florence-quartier-5.geojson", "r") as file:
    bpolys = json.load(file)

parameters = {
    "name": "Currentness",
    "layerName": "",  # TODO
    "bpolys": bpolys,
    "flatten": False,
    "includeSvg": True,
}

for value in parameters.values():
    assert value != "", "Please provide values for the parameters"

response = requests.post(url, json=parameters)
response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

properties = response.json()["properties"]
result = properties["result"]
svg = result.pop("svg")

print("")
print("Result: ")
print(json.dumps(result, indent=4))  # Pretty print response

display(SVG(svg))

AssertionError: Please provide values for the parameters